# Activity 4: Azure CLI - Azure SQL Database

#### <i>The Azure SQL Workshop - Module 2</i>

<p style="border-bottom: 1px solid lightgrey;"></p>

In this activity, you'll walk through getting set up with the Azure CLI, using a PowerShell notebook in ADS.  


**Set up**   

0. You should have opened this file using Azure Data Studio. If you didn't, please refer to Module 2 Activity 3 in the readme.md file to get set up.  
1. In the bar at the top of this screen, confirm or change the "Kernel" to **PowerShell**. This determines what language the code blocks in the file are. In this case, that language is SQL.  
2. You may be prompted to install Python, if you are select **New Python Installation**. This may take a few minutes, you'll see the output in a window that appears at the bottom of ADS.
TODO THIS TAKES ~5 minutes. 
2. Confirming the Kernel is **PowerShell**, for "Attach to", confirm it reads **localhost**.  

Now that you're set up, you should read the text cells and "Run" the code cells by selecting the play button that appears in the left of a code cell when you hover over it.  
> Some of the cells have been run before, this is just to show you the expected result from the testing of the labs. If you choose not to complete the labs/prerequisites, do not run any cells, just review the results.      


In [1]:
# Log in to the Azure portal with your workshop credentials
# You may get an error initially, run again, and you should get a pop-up that directs you through authenticating
az login

In [2]:
# Specify your subscription for the workshop
az account set --subscription <subscription-ID>

In [3]:
# Specify your default subscription, resource group, and Azure SQL Database logical server
az configure --defaults group=azuresqlworkshop0406 sql-server=aw-server0406

Now that you've got your details plugged in, let's see what you can do with the Azure CLI.

In [4]:
# Show all databases in the Azure SQL Database logical server
az sql db list

There's a lot of information above. What if we just want to see specifics for the AdventureWorksID database?
>Note: Replace ID in the command with your unique ID you've been using for the workshop.

In [5]:
az sql db show --name AdventureWorks0406

You can also determine the usage.

In [7]:
az sql db list-usages --name AdventureWorks0406

[
  {
    "currentValue": 29360128.0,
    "displayName": "Database Size",
    "limit": 34359738368.0,
    "name": "database_size",
    "nextResetTime": null,
    "resourceName": "AdventureWorks0406",
    "unit": "Bytes"
  },
  {
    "currentValue": 33554432.0,
    "displayName": "Database Allocated Size",
    "limit": 34359738368.0,
    "name": "database_allocated_size",
    "nextResetTime": null,
    "resourceName": "AdventureWorks0406",
    "unit": "Bytes"
  }
]


You've now seen a few examples for how the `az sql db` commands work. There are also commands related to the Azure SQL Database logical server, and these fall under `az sql server`.  

>There are also similar commands for `az sql mi` and `az sql midb` (commands for databases within a Managed Instance, sometimes referred to as "Managed Databases").  

For detailed explanations of all the commands available, refer to the [Azure CLI documentation](https://docs.microsoft.com/en-us/cli/azure/?view=azure-cli-latest).

## Example: Managing connection policies

One thing thing that you might use the Azure CLI or Azure PowerShell commands for is updating the connection policy. We'll use this as an example for how you can manage Azure SQL using a tool like the Azure CLI. In this example, you'll look at Azure SQL Database and its commands, but the implementation is very similar in Azure SQL Managed Instance.  



### Overview: Connection policies

During deployment, in Azure SQL MI you're able to choose the connection type or policy. In Azure SQL DB, you can choose the connection type, but only via PowerShell or Azure CLI after deployment. You can keep the default (Proxy for connections from outside and Redirect for connections within Azure) or something else. At the highest level, in Proxy mode, all connections are proxied through the Azure SQL Database gateways, but in Redirect mode, after the connection is established leveraging the gateway, the connection is directly to the database or managed instance. The direct connection (redirect) allows for reduced latency and improved throughput, but also requires opening up additional ports (allowing inbound and outbound communication in the range of 11000 - 11999).    

> More information on connectivity for Azure SQL DB can be found [here](https://docs.microsoft.com/en-us/azure/sql-database/sql-database-connectivity-architecture) and for Azure SQL MI [here](https://docs.microsoft.com/en-us/azure/sql-database/sql-database-managed-instance-connectivity-architecture).  

Let's see what the current policy is using the Azure CLI.

In [8]:
az sql server conn-policy show

So the results above tell us the connection type is "Default". What if we want to make everything `Redirect` so we can achieve reduced latency? It's as easy as running two commands.  

First, we need to update the firewall rules to allow the required ports.

In [0]:
az sql server firewall-rule create -n NewRule --start-ip-address 11000 --end-ip-address 11999 # ?????????????????????????????????????????????????????????????

Then, we can update the connection policy

In [0]:
# update policy
az sql server conn-policy update --connection-type Redirect
# confirm update
az sql server conn-policy show